In [29]:
import cv2
import re
import json
import pytesseract
import argparse
import editdistance

In [30]:
def get_text_at_second(t):
    cap.set(cv2.CAP_PROP_POS_MSEC, 1000 * t)
    ret, frame = cap.read()
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    return pytesseract.image_to_string(img_rgb)

In [31]:
def is_same_noisy_text(a, b):
    return editdistance.distance(a.lower(), b.lower()) < 3

In [32]:
def process(file):
    cap = cv2.VideoCapture(file)
    
    fps = cap.get(cv2.CAP_PROP_FPS) # https://stackoverflow.com/a/52032374
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    
    t = 0

    current_text = ''
    new_text = ''


    texts = []

    current_text = get_text_at_second(0)
    current_text_start_pos = 0

    times = list(range(1, int(duration), 30))
    times += list(range(times[-1] + 1, int(duration)))

    for t in range(1, int(duration), 30):
        text = get_text_at_second(t)

        if is_same_noisy_text(text, current_text):
            # same 'slide'
            continue


        found = False

        for j in range(t, current_text_start_pos, -1):
            temp_text = get_text_at_second(j)
            
            if is_same_noisy_text(temp_text, current_text):
                current_text = text
                current_text_start_pos = j
                texts.append((j, text))
                found = True
                break

        if not found:
            print('ERROR: NOT FOUND', t, text)
            break
    
    return texts

In [33]:
def clean_text(text):
    cleaned = text
    cleaned = text.replace('\n', ' ')
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = cleaned.strip()
    
    cleaned = re.sub(r'^Question', '', cleaned, re.I)
    cleaned = re.sub(r'sirimangalo\.org$', '', cleaned, re.I)
    
    cleaned = cleaned.replace('|', 'I')
    
    return cleaned.strip()

In [35]:
%%time
texts = process('./videos/EkFtBW32tDc.mkv')

CPU times: user 4min 3s, sys: 10.4 s, total: 4min 13s
Wall time: 8min 20s


In [19]:
for t, text in texts:
    print('%02d:%02d %s' % (t // 60, t % 60, clean_text(text)))

13:56 How much effort and time should one put into finding a mantra for the experience before it just becomes "thinking"?
16:33 I have an emotional pain that is sinking me. On what basis should I disidentify from it and let it go as not concerning me? It is my responsibility to have created the conditions for it to occur.
24:00 How do we know that our meditation is progressing?
29:00 As I meditate more often, I find my physical experiences during meditation becoming more intense, such as my weight on my feet or pain in my back. Is this normal and will it remain so going forward?
35:47 How do you cope with chronic illness and symptoms of pain with meditation? I find it very challenging as im just getting started with meditation.
38:49 im currently in my last year of studies before university, and there is a lot of pressure for me to go well in studies. I find myself feeling dissatisfied at the end of the day. How can I stay mindful?
43:52 Should one force himself to meditate every day o

In [37]:
for t, text in texts:
    print('%02d:%02d %s' % (t // 60, t % 60, clean_text(text)))

13:56 How much effort and time should one put into finding a mantra for the experience before it just becomes "thinking"?
16:33 I have an emotional pain that is sinking me. On what basis should I disidentify from it and let it go as not concerning me? It is my responsibility to have created the conditions for it to occur.
24:00 How do we know that our meditation is progressing?
29:00 As I meditate more often, I find my physical experiences during meditation becoming more intense, such as my weight on my feet or pain in my back. Is this normal and will it remain so going forward?
35:47 How do you cope with chronic illness and symptoms of pain with meditation? I find it very challenging as im just getting started with meditation.
38:49 im currently in my last year of studies before university, and there is a lot of pressure for me to go well in studies. I find myself feeling dissatisfied at the end of the day. How can I stay mindful?
43:52 Should one force himself to meditate every day o

In [18]:
836 // 60

13

In [36]:
7*30

210

In [25]:
duration

3972.0333333333333